# 1935

Task:

1935 Business census on retail to start, focusing on the tables that summarize the data at the county/city level with establishment count and sales by type of business. 

The first table is on page 88 of 470 (volume III page 82). Some pages are scanned upside down.

Note:

Census File: 
https://archive.org/details/censusofbusiness34unit

Log:

  -2019. Jan. 02 
    
    9:00-12:00 Explore the data page and digitalization.
    
    14:00-19:00 Explore the digitalization problem and solution.
    
    Note: 
        1. Have to manully adjust the orientation of the different pages. The Finereader sometimes cannot tune them correctly.
        2. The raw process is terrible... Serious missing data problem.        
        
    Summary: 
        
        Solved the problem in the Note 1 and 2. The digitalization process should be:
        
        Before process image:
            The image must be rotate manully.
        After import:        
            1. Photo Correction
            2. Crop
            3. Correct Perspective
        
        After this process, we could generate excel file free from most format error.
   
   -2019. Jan. 04
    
    9:00-12:00 13:00-19:30: Examine the digitalization problem and solution.
    
    Note: 
        1. The above process works well, we have generated fairly good result of the digitalization.
        2. The distortion of image result in some problem during the OCR and formatting process. And there is no existing OCR solution to correct bending line to straight.
        3. The FineReader cannot automaticlly correct the perspective.
        
    Summary: 
        1. Standardize the above process applies to other formes.
        2. Hardly to improve with other operation.
        
   
   -2019. Jan. 05-06
   
    Task: improve the performance
    Question: PDF loss some information?
    -2019. Jan. 05 9:00-12:00 13:00-19:30: Study the influence of raw file formats and size (jp2,jpeg,png,pdf,xml to excel,xml)
    -2019. Jan. 06 9:00-12:00 13:00-18:00: Study the conversion and preprocess of jp2 files.
    
    Image converter website: https://www.freefileconvert.com/file/Rmr8R5Jo0LlO,aB-601EO2pk6,21-Mm8B75-4O,kjpRD2jqe-n0
    
    Conclusion:
        Image (jp2) do have better performance.
        But except the format conversion process. We need to manully process the image.
        Also, the format conversion is not available for mac, since the pillow (PIL) does not support jpegopen package on mac.
        
    So, our image preprocessing process is finalized.

# Code
# Prepare raw df list

In [1]:
import pandas as pd
import numpy as np

from PIL import Image

from os import listdir
from os.path import isfile, join
from pandas import ExcelWriter

In [2]:
def report_state(state_raw_df):
    for state in US_states:
        for string in state_raw_df[0]:
            try:
                if state.lower() in string.lower(): 
                    return state
            except:
                pass
    return None

In [3]:
US_states=['Alabama','Alaska','Arizona','Arkansas','California','Colorado',
        'Connecticut','Delaware','District of Columbia','Florida','Georgia','Hawaii','Idaho', 
        'Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana',
        'Maine' 'Maryland','Massachusetts','Michigan','Minnesota',
        'Mississippi', 'Missouri','Montana','Nebraska','Nevada',
        'New Hampshire','New Jersey','New Mexico','New York',
        'North Carolina','North Dakota','Ohio',    
        'Oklahoma','Oregon','Pennsylvania','Rhode Island',
        'South  Carolina','South Dakota','Tennessee','Texas','Utah',
        'Vermont','Virginia','Washington','West Virginia',
        'Wisconsin','Wyoming']

In [4]:
file_path='./Data/Intermediate/censusofbusiness34unit_jp2_88_285_mod_180119_modified.xlsx/'
file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
file_names.sort()
#https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory

In [5]:
#file_name
#pd.read_excel(file_path+'censusofbusiness34unit_jp2_88_285_mod_180119_modified - 0023.xlsx',index_col=0)

In [6]:
form_raw_list=[]
for file_name in file_names:
    if file_name[0]!='~':# exclude temp file name
        #print(file_name) # to check error
        form_raw_list.append(pd.read_excel(file_path+file_name,header=None,index_col=0))

In [7]:
#pd.read_excel('./Data/Intermediate/censusofbusiness34unit_jp2_88_285_mod_180119_modified.xlsx/'+'censusofbusiness34unit_jp2_88_285_mod_180119_modified - 0007.xlsx',header=None, skiprows=0,index_col=0)

In [8]:
col_num=[len(df.columns.get_values()) for df in form_raw_list]
set(col_num) #24
#col_num.index(25) # modify the wrong ocr ed frame

{24}

In [9]:
form_raw_list[0].index[3]

'Autauga County'

In [10]:
def report_start_line(df):
    idx=df.index
    for i in range(len(idx)):
        if pd.notnull(idx[i]) and 'county' in idx[i].lower():
            return i
    return 'o'

In [11]:
form_start_line=[report_start_line(df) for df in form_raw_list]

In [12]:
set(form_start_line)

{2, 3, 4, 5, 'o'}

In [13]:
for i in range(len(form_start_line)):
    if form_start_line[i]=='o': print(i+1)

20
66
67
68
69


In [14]:
#pd.read_excel('./Data/Intermediate/censusofbusiness34unit_jp2_88_285_mod_180119_modified.xlsx/'+'censusofbusiness34unit_jp2_88_285_mod_180119_modified - 0069.xlsx',header=None,index_col=0)
#20: [3:]
#66:[6:]
#67: [3:]
#68:[3:]
#69:[3:]

for form_name, starter in zip([20,66,67,68,69],[3,6,3,3,3]):
    form_start_line[form_name-1]=starter

In [15]:
set(form_start_line)

{2, 3, 4, 5, 6}

In [16]:
for i in range(len(form_raw_list)):
    form_raw_list[i]=form_raw_list[i].iloc[form_start_line[i]:]

In [17]:
form_raw_list[0].head(3)

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
0,,,,,,,,,,,,,,,,,,,,,
Autauga County,138,1121,37,165,8,11,44,529,3,36,...,19,93,3,z,4,42,5,47,4,51
Baldwin County,339,3175,95,723,22,58,46,589,10,92,...,73,280,14,138,13,285,11,129,24,168
Barbour County:,249,2.843,81,724,17,41,35,320,13,173,...,43,226,5,198,9,168,11,115,15,127


# Modify Raw df list

In [18]:
#pd.read_excel(file_path+'censusofbusiness34unit_jp2_88_285_mod_180119_modified - 0144.xlsx',header=None, skip_rows=2,index_col=0)

In [19]:
# reconstruct columns
eg_form_process=pd.read_excel(file_path+file_names[0],header=[0,1])

# fix the columns

eg_columns=eg_form_process.columns.get_values()

eg_columns=[list(tuple_) for tuple_ in eg_columns]

for i in range(0,int(len(eg_columns)/2)):
    eg_columns[2*i][1]='Store Numbers'
    eg_columns[2*i+1][1]='Sales'
    if eg_columns[2*i][0]==eg_columns[2*i+1][0]:
        temp=eg_columns[2*i][0]
    else:
        temp=eg_columns[2*i][0]+' '+eg_columns[2*i+1][0]   
    
    temp=temp.replace('-','')
    
    eg_columns[2*i][0]=temp
    eg_columns[2*i+1][0]=temp

eg_columns=[tuple(list_) for list_ in eg_columns]
# fix the columns in the df list
for df in form_raw_list:
    df.columns=pd.MultiIndex.from_tuples(eg_columns)


In [34]:
eg_form_process.columns=pd.MultiIndex.from_tuples(eg_columns)
eg_form_process.head(3)

Total             Food stores          Eating places  \
               Store Numbers     Sales Store Numbers    Sales Store Numbers   
STATE TOTAL            22167  337217.0          7485  74814.0          0.19   
Autauga County           138    1121.0            37    165.0          8.00   
Baldwin County           339    3175.0            95    723.0         22.00   

                      General stores (with food)         \
                Sales              Store Numbers  Sales   
STATE TOTAL     12403                      3.138  46617   
Autauga County     11                         44    529   
Baldwin County     58                         46    589   

               General merchandise group            ...      Filling stations  \
                           Store Numbers  Sales     ...         Store Numbers   
STATE TOTAL                          788  43548     ...                 3.052   
Autauga County                         3     36     ...                19.000   
Baldwin County                        10     92     ...                73.000   

                      Furniture household radio         \
                Sales             Store Numbers  Sales   
STATE TOTAL     21439                       554  14876   
Autauga County     93                         3      z   
Baldwin County    280                        14    138   

               Lumber building hardware          Drug stores         \
                          Store Numbers  Sales Store Numbers  Sales   
STATE TOTAL                         551  15.48           836  14882   
Autauga County                        4     42             5     47   
Baldwin County                       13    285            11    129   

                Other stores               
               Store Numbers        Sales  
STATE TOTAL            1.569  M • <7» o H  
Autauga County         4.000           51  
Baldwin County        24.000          168  

[3 rows x 24 columns]

In [21]:
#len(eg_form_process.columns) 24 #checkex

In [22]:
# start to extract state
state_file_path='./Data/Intermediate/censusofbusiness34unit_jp2_88_285_mod_180119_modified copy.xlsx/'
state_file_names = [f for f in listdir(state_file_path) if isfile(join(state_file_path, f))]
state_file_names.sort()

In [23]:
state_raw_list=[]
for state_file_name in state_file_names:
    if state_file_name[0]!='~':# exclude temp file name
        state_raw_list.append(pd.read_excel(state_file_path+state_file_name,header=None,skiprows=None,nrows=10))

In [24]:
state_extract_list=[]
for df in state_raw_list:
    state_extract_list.append(report_state(df))

In [25]:
# forward filling
for i in range(1,len(state_extract_list)):
    if state_extract_list[i]==None: state_extract_list[i]=state_extract_list[i-1]

In [26]:
for form,state in zip(form_raw_list,state_extract_list):
    form['State']=state

In [33]:
form_raw_list[0].head(3)

Total          Food stores       Eating places        \
                Store Numbers  Sales Store Numbers Sales Store Numbers Sales   
0                                                                              
Autauga County            138   1121            37   165             8    11   
Baldwin County            339   3175            95   723            22    58   
Barbour County:           249  2.843            81   724            17    41   

                General stores (with food)       General merchandise group  \
                             Store Numbers Sales             Store Numbers   
0                                                                            
Autauga County                          44   529                         3   
Baldwin County                          46   589                        10   
Barbour County:                         35   320                        13   

                        ...    Filling stations Furniture household radio  \
                Sales   ...               Sales             Store Numbers   
0                       ...                                                 
Autauga County     36   ...                  93                         3   
Baldwin County     92   ...                 280                        14   
Barbour County:   173   ...                 226                         5   

                      Lumber building hardware         Drug stores        \
                Sales            Store Numbers Sales Store Numbers Sales   
0                                                                          
Autauga County      z                        4    42             5    47   
Baldwin County    138                       13   285            11   129   
Barbour County:   198                        9   168            11   115   

                 Other stores          State  
                Store Numbers Sales           
0                                             
Autauga County              4    51  Alabama  
Baldwin County             24   168  Alabama  
Barbour County:            15   127  Alabama  

[3 rows x 25 columns]

In [28]:
df_output=pd.concat(form_raw_list)

In [32]:
df_output.head(3)

Total          Food stores       Eating places        \
                Store Numbers  Sales Store Numbers Sales Store Numbers Sales   
0                                                                              
Autauga County            138   1121            37   165             8    11   
Baldwin County            339   3175            95   723            22    58   
Barbour County:           249  2.843            81   724            17    41   

                General stores (with food)       General merchandise group  \
                             Store Numbers Sales             Store Numbers   
0                                                                            
Autauga County                          44   529                         3   
Baldwin County                          46   589                        10   
Barbour County:                         35   320                        13   

                        ...    Filling stations Furniture household radio  \
                Sales   ...               Sales             Store Numbers   
0                       ...                                                 
Autauga County     36   ...                  93                         3   
Baldwin County     92   ...                 280                        14   
Barbour County:   173   ...                 226                         5   

                      Lumber building hardware         Drug stores        \
                Sales            Store Numbers Sales Store Numbers Sales   
0                                                                          
Autauga County      z                        4    42             5    47   
Baldwin County    138                       13   285            11   129   
Barbour County:   198                        9   168            11   115   

                 Other stores          State  
                Store Numbers Sales           
0                                             
Autauga County              4    51  Alabama  
Baldwin County             24   168  Alabama  
Barbour County:            15   127  Alabama  

[3 rows x 25 columns]

In [30]:
out_put_year=1935
file_path='./Data/Processed/'
writer = ExcelWriter(file_path+str(out_put_year)+'.xlsx')
df_output.to_excel(writer)
writer.save()

In [31]:
#These code should be working with windows system.
#for image_name in image_names:
#    img = Image.open(image_path+'/'+image_name) # Input Image
#    img.save(image_path+'/'+image_name[:-4]+'.jpeg') 